In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [2]:
# set up splinter and the executable path. 
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/tobix/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


### Visit the Mars NASA news Site

In [3]:
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')


In [5]:
slide_elem.find('div', class_='content_title')


<div class="content_title">NASA's Perseverance Rover Will Look at Mars Through These 'Eyes'</div>

In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's Perseverance Rover Will Look at Mars Through These 'Eyes'"

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p


'A pair of zoomable cameras will help scientists and rover drivers with high-resolution color images.'

### Images Featured


In [8]:
url = 'https://spaceimages-mars.com'
browser.visit(url)


In [9]:
#find and click the full image button

full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [10]:
# parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')


In [11]:
# find the relative image

img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [12]:
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

### Mars Facts

In [13]:
url = 'https://galaxyfacts-mars.com'
browser.visit(url)

In [14]:
# html = browser.html
# mars_facts_soup = soup(html, 'html.parser')

df = pd.read_html(url)[0]
df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [15]:
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace = True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [16]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

### Scrape High-Resolution Mars' Hemisphere Images and Titles

In [17]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)


In [18]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.


for x in range(0,4):
    browser.find_by_tag('h3')[x].click()
    time.sleep(2)


    html = browser.html
    hemisphere_soup = soup(html, 'html.parser')

    full_hemi = hemisphere_soup('div', class_='container')
    

    for each_hemi in full_hemi:

        hemispheres = {}
        
        
        hemi_urls_rel = each_hemi.find('img', class_='wide-image').get('src')
        
        hemi_urls = f'https://astrogeology.usgs.gov/cache/{hemi_urls_rel}'
        
        hemi_titles = each_hemi.h2.text

        hemispheres = {'img_url':hemi_urls, 'title': hemi_titles}

        hemisphere_image_urls.append(hemispheres)

    browser.back()
    
    time.sleep(2)
    




In [19]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [20]:
# 5. Quit the browser
browser.quit()


SyntaxError: invalid syntax (2416188498.py, line 1)